In [9]:
!conda update conda
#pip3 install --user --upgrade tensorflow
#!pip install tensorflow

Solving environment: failed

NotWritableError: The current user does not have write permissions to a required path.
  path: /home/ubuntu/anaconda2/pkgs/urls.txt
  uid: 1000
  gid: 1000

If you feel that permissions on this path are set incorrectly, you can manually
change them by executing

  $ sudo chown 1000:1000 /home/ubuntu/anaconda2/pkgs/urls.txt

In general, it's not advisable to use 'sudo conda'.




In [8]:
import pandas as pd
notes = pd.read_csv('demo.csv')

notes = notes.groupby('HADM_ID')
yo = 0
for key, val in notes:
    k = val.iloc[0]['TEXT']
    k = k.split(': ')
    [print(i) for i in k]
    break

PATIENT/TEST INFORMATION:
Indication
Left ventricular function. Pericardial effusion. Right ventricular function.
Height
(in) 71
Weight (lb)
190
BSA (m2)
2.07 m2
BP (mm Hg)
108/56
HR (bpm)
84
Status
Inpatient
Date/Time
[**2150-4-18**] at 12:36
Test
Portable TTE (Complete)
Doppler
Full Doppler and color Doppler
Contrast
None
Technical Quality
Adequate


INTERPRETATION:

Findings:

LEFT ATRIUM
Dilated LA.

RIGHT ATRIUM/INTERATRIAL SEPTUM
Mildly dilated RA.

LEFT VENTRICLE
Normal LV wall thickness and cavity size. Hyperdynamic LVEF
>75%. No resting LVOT gradient.

RIGHT VENTRICLE
Normal RV chamber size and free wall motion.

AORTA
Normal aortic diameter at the sinus level. Normal ascending aorta
diameter.

AORTIC VALVE
Mildly thickened aortic valve leaflets (3). No AS. No AR.

MITRAL VALVE
Mildly thickened mitral valve leaflets. Mild (1+) MR. Normal LV
inflow pattern for age.

TRICUSPID VALVE
Normal tricuspid valve leaflets. Moderate PA systolic
hypertension.

PERICARDIUM
No pericardial e

In [424]:
hey = preprocess_notes(ex, 500)


,SUBJECT_ID,HADM_ID,CHARTDATE,NOTE
16928,54610,100003.0,2150-04-18,The left atrium is dilated. Left ventricular w...


,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,NOTE
16928,76581,76338,54610,100003.0,2150-04-18,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Left ve...,apo


In [425]:
def preprocess_notes(vst, max_len):
    vst = vst.sort_values('CHARTDATE')  #arrange by day1,day2, day3..
    #input VST is a dataframe of all rows in each individual visit
    # return a new series with shorter sentences but corresponds to each day during this visit
    df = vst['TEXT'] ##extract "TEXT" column as a series
    df = df.str.replace('\n', ' ')
    df = df.str.replace('  ', ' ')
    df = df.str.replace('\*+', '')
    df = df.str.replace('\d+', '')
    df = df.str.strip("[]")
    newcol = []
    for row in df:
        short = ''
        row = row.split(': ')
        row.sort(key=len)
        while len(short) < max_len:
            short += row.pop()
        short = short[0:max_len]
        newcol.append(short)
    newcol = pd.DataFrame({'NOTE': newcol}) 
    
    result = vst[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE']]
    newcol= newcol.set_index(result.index)
    
    #newcol.apply(lambda x:str(x))
    #print(type(newcol.iloc[0]))
    result = pd.concat([result, newcol], axis=1)
    return result

In [12]:
type(visits[10])

pandas.core.frame.DataFrame

In [9]:
work = pd.read_csv('processed.csv')

In [27]:
work.head()
strArray = work["NOTE"].values

str

In [31]:
import os
import json

In [34]:
def get_embedding(sentence):  #work on each string/row of note
    outF = open("prepinput.txt", "w")
    outF.write(sentence)
    outF.close()
    
    out = open('prepoutput.jsonl', 'w')
    
    os.system('python3 ~/payload/bioBertPart/biobert-master/bioextract_features.py \
    --input_file=~/payload/bioBertPart/prepinput.txt \
    --vocab_file=~/payload/bioBertPart/pubmed_pmc_470k/vocab.txt \
    --bert_config_file=~/payload/bioBertPart/pubmed_pmc_470k/bert_config.json \
    --init_checkpoint=~/payload/bioBertPart/pubmed_pmc_470k/biobert_model.ckpt \
    --output_file=~/payload/bioBertPart/prepoutput.jsonl')
    
    with open('prepoutput.jsonl') as f:
        d = json.load(f)
    os.system('rm prepinput.txt')
    os.system('rm prepoutput.txt')
    
    sentence_vector = []
    for i in range (1, len(d['features'])-1):
        sentence_vector = sentence_vector + d['features'][i]['layers'][0]['values']
        
    number_of_tokens = len(d['features']) - 2
    for elem in sentence_vector:
        elem = elem/number_of_tokens
    return sentence_vector

In [35]:
embedding = []
for stce in strArray:
    embedding.append(get_embedding(stce))
    break

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
embedding.to_csv(r'embedding.csv')